In [1]:
#!pip install selenium --quiet

In [2]:
#!pip install beautifulsoup4 --quiet

In [3]:
#!pip3 install webdriver-manager --quiet

In [4]:
location_tosearch = "India"              #LOCATION
job_category= "Data Science"             #JOB CATEGORY 

In [5]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By


In [6]:
# Open browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Opening linkedin/jobs website
driver.get("https://in.linkedin.com/jobs")
## waiting load the webpage , can change depending on internet speed.
time.sleep(2)

#Maximizing browser window to avoid hidden elements
driver.set_window_size(1024, 600)
driver.maximize_window()

In [7]:



#type job_category in search field
job_search_element_name = "keywords"
driver.find_element(By.NAME, job_search_element_name).send_keys(job_category)


#type location in location field
delete_current_location = driver.find_element(By.XPATH,"//*[@id=\"JOBS\"]/section[2]/button/icon").click()
location_element = driver.find_element(By.XPATH, "//*[@id=\"JOBS\"]/section[2]/input")
location_element.send_keys(location_tosearch)


In [8]:
#find the job-search button!
button_xpath = "//*[@id=\"main-content\"]/section[1]/div/section/div[2]/button[2]"
button_element = driver.find_element(By.XPATH, button_xpath)

#click search button!
webdriver.ActionChains(driver).move_to_element(button_element ).click(button_element ).perform()


# time to load the page
#can be lowered depdending on computer,internet speed
time.sleep(3)


In [9]:
#scrolling the page

#Can add EC wait here and check for "View More Jobs" button , or click on button and get more results .

def scroll_to_max(driver):
    SCROLL_PAUSE_TIME = 5 #5 seconds
    time.sleep(SCROLL_PAUSE_TIME)

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    for x in range(0, 10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.find_element(By.XPATH, "//body").send_keys(Keys.ARROW_UP)
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        #print('current Y: ' + str(new_height))
        if new_height == last_height:
            break
        last_height = new_height

scroll_to_max(driver)

In [10]:
#empty data-structures to save data
visited_url = set () #check in constant time
job_details_dict = {}

#fuction to save the extracted data
def save_data(clean_company_url, company_name, job_location, job_role,description,employee):
    job_details_list = [company_name, job_location, job_role,description,employee]
    job_details_dict[clean_company_url] = job_details_list
    print(company_name)

In [11]:
#Function to extract post urls from job search page and save in a list.
job_post_urls = []
def extract_job_post_url(driver):
    page_source = BeautifulSoup(driver.page_source)
    job_post = page_source.find_all("a", class_ ="base-card__full-link")
    for url in job_post:
        url = url.get("href")
        clean_url = url.split("?")[0]
        job_post_urls.append(clean_url)
        

#Function call
extract_job_post_url(driver)    

In [12]:
#Function - Extracts Job Info FROM URL
def extractor_l1(url):
    while True:

        print("Visiting  -> Job Post Page")
        job_page = driver.get(url)
        time.sleep(2)
        current_url = driver.current_url
        if current_url == url:

            #Getting jobrole!

            job_role_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[2]/div/div[1]/div/h1"
            job_role_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                (By.XPATH, job_role_xpath)))
            job_role = (job_role_element.get_attribute('innerHTML')).strip()

            #Getting job location!

            job_location_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[2]"
            job_location_element =  WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                (By.XPATH, job_location_xpath)))
            job_location = (job_location_element.get_attribute('innerHTML')).strip()

            #Getting company name and url 

            company_name_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a"
            company_name_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                (By.XPATH, company_name_xpath)))
            company_name = (company_name_element.get_attribute('innerHTML')).strip()
            company_url = company_name_element.get_attribute('href')
            clean_company_url =  company_url.split("?")[0]

            #Adding the url to visited url                                                     
            visited_url.add(url)

            return (job_role,job_location,company_name,clean_company_url)
        
        else:
            print("Login Requested- retrying In different tab")
            time.sleep(2)
            driver.switch_to.new_window('tab')
            driver.switch_to.window(driver.window_handles[0])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])



In [16]:

#Function EXtract Info from Company Page (Description and Employee)
def get_description(url,company_name):
    counter = 0
    while True:
    
        
        print("Visiting  -> %s")%(company_name)
        
        company_page = driver.get(url)
        time.sleep(2)
        current_url = driver.current_url
        
        if current_url == url:
             
            try:
                
                desc_xpath = "//*[@id=\"main-content\"]/section[1]/div/section[1]/div/p"
                desc_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
                    (By.XPATH, desc_xpath,)))
                
                description = (desc_element.get_attribute('innerHTML')).strip()
                
                employee_expath = "//*[@id=\"main-content\"]/section[1]/section/div/div[2]/div[2]/ul/li/div/a"
                employee_element = driver.find_element(By.XPATH, employee_expath)
                employee = (employee_element.get_attribute('innerHTML')).strip()
                employee = ''.join(filter(lambda i: i.isdigit(), employee))
                
                return description,employee

            except TimeoutException as timeout:
                print("Failed")
                description = "n/a"
                employee = "n/a"
                return (description, employee)
        else:
            
            print("Login Requested- retrying In different tab")
            time.sleep(10)
            driver.switch_to.new_window('tab')
            driver.switch_to.window(driver.window_handles[0])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)
            
            
    
    
    
    

In [17]:
#Taking all the data and calling the save function!

def final_step(job_role,job_location,company_name,clean_company_url):
    description,employee = get_description(clean_company_url,company_name)
    save_data(clean_company_url, company_name, job_location, job_role, description,employee)

In [18]:
#Get Data from Job Post Urls:

for url in job_post_urls:
    if url in visited_url:
        print("already visited")
        continue
    job_role,job_location,company_name,clean_company_url = (extractor_l1(url))
    final_step(job_role,job_location,company_name,clean_company_url)

    
    
  

already visited
Visiting  -> Job Post Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Everi Holdings Inc.
Visiting  -> Job Post Page
Visiting  -> Company Page
Corizo
Visiting  -> Job Post Page
Visiting  -> Company Page
SkillVertex
Visiting  -> Job Post Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page
Visiting  -> Company Page


KeyboardInterrupt: 